In [ ]:
! pip install -q transformers
! pip install -q datasets
! pip install -q huggingface_hub
! pip install -q sentencepiece
! pip install -q evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import json

from datasets import load_dataset, load_from_disk, Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModel, pipeline, TrainingArguments, Trainer
from tqdm import tqdm
from IPython.display import clear_output
import torch
from huggingface_hub import notebook_login
import evaluate
from google.colab import drive
import pandas as pd

In [ ]:
model_checkpoint = "bert-base-multilingual-uncased"
max_length = 512
doc_stride = 256
batch_size = 8
lr = 3e-5
epoch = 8

In [ ]:
MODEL_PATH = "./"

In [ ]:
org_dataset = load_dataset("Hamid-reza/small-persian-QA")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1261 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/130 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/Hamid-reza___parquet/Hamid-reza--small-persian-QA-0ef749d72b9649a4/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/130 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/mohammadhossein___parquet/mohammadhossein--addany-dataset-d9f1e2d06d1b9be9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
org_train_dataset = org_dataset["train"]
org_val_dataset = org_dataset["validation"]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,)
    
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
org_tokenized_train = org_train_dataset.map(prepare_train_features, batched=True, remove_columns=org_train_dataset.column_names)
org_tokenized_val = org_val_dataset.map(prepare_train_features, batched=True, remove_columns=org_val_dataset.column_names)

Map:   0%|          | 0/1261 [00:00<?, ? examples/s]

Map:   0%|          | 0/130 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/130 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-b

In [ ]:
args = TrainingArguments(
    MODEL_PATH,
    evaluation_strategy = "epoch",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=0.0001,
    save_strategy="no") 

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=org_tokenized_train,
    eval_dataset=org_tokenized_val,
    tokenizer=tokenizer)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1275
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1280
  Number of trainable parameters = 166767362
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,2.642254
2,No log,2.464851
3,No log,2.805408
4,2.506400,3.190945
5,2.506400,3.486594
6,2.506400,3.950189
7,0.540500,4.078456
8,0.540500,4.229901


***** Running Evaluation *****
  Num examples = 130
  Batch size = 8
***** Running Evaluation *****
  Num examples = 130
  Batch size = 8
***** Running Evaluation *****
  Num examples = 130
  Batch size = 8
***** Running Evaluation *****
  Num examples = 130
  Batch size = 8
***** Running Evaluation *****
  Num examples = 130
  Batch size = 8
***** Running Evaluation *****
  Num examples = 130
  Batch size = 8
***** Running Evaluation *****
  Num examples = 130
  Batch size = 8
***** Running Evaluation *****
  Num examples = 130
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1280, training_loss=1.228011167049408, metrics={'train_runtime': 928.7884, 'train_samples_per_second': 10.982, 'train_steps_per_second': 1.378, 'total_flos': 2145644442080400.0, 'train_loss': 1.228011167049408, 'epoch': 8.0})

In [ ]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
model.push_to_hub("mbert_v1")
tokenizer.push_to_hub("mbert_tokenizer_v1")

Configuration saved in /tmp/tmpcpv4h2g5/config.json
Model weights saved in /tmp/tmpcpv4h2g5/pytorch_model.bin
Uploading the following files to arashmarioriyad/addany_mbert_v1: config.json,pytorch_model.bin


Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

tokenizer config file saved in /tmp/tmpvy81s51t/tokenizer_config.json
Special tokens file saved in /tmp/tmpvy81s51t/special_tokens_map.json
Uploading the following files to arashmarioriyad/addany_mbert_tokenizer_v1: vocab.txt,tokenizer_config.json,special_tokens_map.json,tokenizer.json


CommitInfo(commit_url='https://huggingface.co/arashmarioriyad/addany_mbert_tokenizer_v1/commit/5ddc9d9e6cdc9523be7d58d3aa5970f2abe68df1', commit_message='Upload tokenizer', commit_description='', oid='5ddc9d9e6cdc9523be7d58d3aa5970f2abe68df1', pr_url=None, pr_revision=None, pr_num=None)